In [ ]:
# Importamos bibliotecas
import requests
from lxml import html
import time
import random
import pandas as pd

In [ ]:
#macros, se podren pasar como argumentos en futuras versiones
#hasta = '2021-12-31'
#desde = '2021-01-01'
# area = 'advantage actor critic'
# fuente = 'arxiv'
# numero de paginas a revisar (paginas de 10 elementos)
num_de_paginas = 2


In [ ]:
info_fuentes = []

info_fuente_arvix = {
    'area':'advantage actor critic',
    'fuente':'arxiv',
    'titulo':'//h1 [@class ="title mathjax"]/text()',
    'autores':'//div [@class ="authors"]/a/text()',
    'abstract':'//blockquote [@class ="abstract mathjax"]/text()',
    'clase_primaria':'//span [@class ="primary-subject"]/text()',
    # 'otras_clases':'//td [@class ="tablecell subjects"]/text()',
}
info_fuente_arvix['pagina'] = 'https://scholar.google.com/scholar?q='+info_fuente_arvix.get("area").replace(" ", "+")+'+source%3A'+info_fuente_arvix.get("fuente")+'&oq='
info_fuentes.append(info_fuente_arvix)

info_fuente_springer = {
    'area':'advantage actor critic',
    'fuente':'springer',
    'titulo':'//h1 [@class ="c-article-title"]/text()',
    'autores':'//li [@class ="c-article-author-list__item"]/a/text()',
    'abstract':'//div [@class ="c-article-section__content"][@id="Abs1-content"]/p/text()',
    'clase_primaria':'//li [@class ="c-article-subject-list__subject"]/span/text()',
    # 'otras_clases':'',
}
info_fuente_springer['pagina'] = 'https://scholar.google.com/scholar?q='+info_fuente_springer.get("area").replace(" ", "+")+'+source%3A'+info_fuente_springer.get("fuente")+'&oq='
info_fuentes.append(info_fuente_springer)

In [ ]:
for k in range(len(info_fuentes)):
    headers = requests.utils.default_headers()

    headers.update(
        {
            'User-Agent': 'Chrome/96.0.4664.93',
        }
    )

    req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)
    if req.status_code == 429:
        print('wait' +req.headers.get('Retry-After'))
        time.sleep(int(req.headers.get('Retry-After'))+1)
        req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)
    print(req)

    #Convertimos a tree 
    arbol_html = html.fromstring(req.content)

    # start=10 

    papers = []


    for i in range (num_de_paginas):
        print('Empiezo vuelta '+str(i)+' :'+time.strftime("%H:%M:%S", time.gmtime()))
        req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)
        print('scrapig google', info_fuentes[k].get('pagina'))

        # repetir despues del tiempo de espera
        if req.status_code == 429:
            print('wait' +req.headers.get('Retry-After'))
            time.sleep(int(req.headers.get('Retry-After'))+1)
            req =  requests.get(info_fuentes[k].get('pagina'), headers=headers)

        # Convertimos a tree 
        arbol_html = html.fromstring(req.content)
        temp_res = arbol_html.xpath('//h3 [@class ="gs_rt"]/a/@href')

        # for j in range(len(temp_res)):
        for j in range(1):
            paper_dic = {}
            print('scraping a', temp_res[j])
            pag = temp_res[j]
            time.sleep(random.randint(1,2)*0.5)
            paper_res =  requests.get(pag, headers=headers)
            
            if paper_res.status_code == 429:
                print('wait' +paper_res.headers.get('Retry-After'))
                time.sleep(int(paper_res.headers.get('Retry-After'))+1)
                paper_res =  requests.get(pag, headers=headers)
            paper_arbol_html = html.fromstring(paper_res.content)

            # especifico de la pagina
            paper_dic['fuente'] = info_fuentes[k]['fuente']
            paper_dic['titulo'] = paper_arbol_html.xpath(info_fuentes[k]['titulo'])
            paper_dic['autores'] = paper_arbol_html.xpath(info_fuentes[k]['autores'])
            paper_dic['abstract'] = paper_arbol_html.xpath(info_fuentes[k]['abstract'])
            paper_dic['clase_primaria'] = paper_arbol_html.xpath(info_fuentes[k]['clase_primaria'])
            # paper_dic['otras_clases'] = paper_arbol_html.xpath(info_fuentes[k]['otras_clases'])
            #metadata
            paper_dic['pag_espec'] = pag
            paper_dic['pag_espec'] = info_fuentes[k].get('pagina')
            #generico
            paper_dic["n_citaciones"] = arbol_html.xpath('//a [@href ="'+pag+'"]/parent::*/parent::*/div [@class = "gs_fl"]/a [contains(text(),"Citado")]/text()')
            paper_dic["citado"] = arbol_html.xpath('//a [@href ="'+pag+'"]/parent::*/parent::*/div [@class = "gs_fl"]/a [contains(text(),"Citado")]/@href')
            papers.append(paper_dic)
        info_fuentes[k]['pagina'] = 'https://scholar.google.com/scholar?start='+str(10*(i+1))+'&q='+info_fuentes[k].get("area").replace(" ", "+")+'+source%3A'+info_fuentes[k].get("fuente").split('.')[0]+'&oq='
        

    
    
    


In [ ]:
for paper in papers:
    print(paper)
    break

In [ ]:
for i in papers:
    print(i["titulo"])

In [ ]:
for i in papers:
    print(i["titulo"])
for i in papers:
    print(i["autores"])
for i in papers:
    print(i["abstract"])
for i in papers:
    print(i["clase_primaria"])
for i in papers:
    print(i["n_citaciones"])